In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
import csv
import re

def scrape_books():
    """Scrape book data from books.toscrape.com with robust price extraction"""
    url = "https://books.toscrape.com/"
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.find_all('article', class_='product_pod')[:10]
        
        book_data = []
        for book in books:
            title = book.h3.a['title']
            price_element = book.find('p', class_='price_color')
            
            # Extract price using regex to find numbers
            price_text = price_element.text
            price_match = re.search(r'[\d\.]+', price_text)
            
            if price_match:
                price_value = float(price_match.group())
            else:
                # Fallback: try to extract from the HTML directly
                price_html = str(price_element)
                price_match = re.search(r'[\d\.]+', price_html)
                price_value = float(price_match.group()) if price_match else 0.0
            
            book_data.append({
                'title': title,
                'price_gbp': price_value,
                'scraped_at': datetime.now().isoformat()
            })
            
        return book_data
        
    except Exception as e:
        print(f"Error during scraping: {e}")
        return []

def post4csv_convert(amount, from_currency, to_currency):
    """
    Convert currency using Post4CSV API with the provided API key
    """
    try:
        # Post4CSV API endpoint (replace with actual endpoint if different)
        api_url = "https://api.post4csv.com/convert"
        
        # Prepare the request parameters
        params = {
            'amount': amount,
            'from': from_currency,
            'to': to_currency,
            'apikey': '98f1bd1273c4e5e295f59f5a'  # Your API key
        }
        
        # Make the API request
        response = requests.get(api_url, params=params, timeout=10)
        response.raise_for_status()
        
        # Parse the response
        data = response.json()
        
        # Extract the converted amount (adjust based on actual API response structure)
        if 'result' in data and 'converted_amount' in data['result']:
            return round(float(data['result']['converted_amount']), 2)
        elif 'converted_amount' in data:
            return round(float(data['converted_amount']), 2)
        else:
            print(f"Unexpected API response format: {data}")
            # Fallback to mock rates if API response format is unexpected
            return post4csv_fallback(amount, from_currency, to_currency)
            
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        # Fallback to mock rates if API is unavailable
        return post4csv_fallback(amount, from_currency, to_currency)
    except Exception as e:
        print(f"Error converting currency: {e}")
        return post4csv_fallback(amount, from_currency, to_currency)

def post4csv_fallback(amount, from_currency, to_currency):
    """Fallback conversion using mock rates if API fails"""
    print(f"Using fallback conversion for {amount} {from_currency} to {to_currency}")
    
    mock_rates = {
        'GBP': {'USD': 1.2, 'EUR': 1.1, 'KES': 150},
        'USD': {'GBP': 0.83, 'EUR': 0.85, 'KES': 110},
        'EUR': {'GBP': 0.9, 'USD': 1.18, 'KES': 130},
        'KES': {'GBP': 0.0067, 'USD': 0.0091, 'EUR': 0.0077}
    }
    
    if from_currency in mock_rates and to_currency in mock_rates[from_currency]:
        return round(amount * mock_rates[from_currency][to_currency], 2)
    else:
        return amount  # Return original if conversion not available

def save_to_csv(books_data, filename="converted_prices.csv"):
    """Save book data to CSV file"""
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['title', 'price_gbp', 'price_converted', 'currency', 'scraped_at', 'converted_at']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for book in books_data:
                writer.writerow(book)
                
        print(f"Data saved to {filename}")
        return True
        
    except Exception as e:
        print(f"Error saving to CSV: {e}")
        return False

def save_to_json(books_data, filename="converted_prices.json"):
    """Save book data to JSON file"""
    try:
        with open(filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(books_data, jsonfile, indent=4)
            
        print(f"Data saved to {filename}")
        return True
        
    except Exception as e:
        print(f"Error saving to JSON: {e}")
        return False

def display_results(books_data, target_currency):
    """Display the results in a formatted table"""
    print(f"\n{'Book Title':<50} {'Price (GBP)':<12} {'Price (' + target_currency + ')':<15}")
    print("-" * 80)
    
    for book in books_data:
        title = book['title'][:47] + "..." if len(book['title']) > 47 else book['title']
        print(f"{title:<50} £{book['price_gbp']:<11.2f} {book['price_converted']:<14.2f}")

def main():
    print("Scraping book prices from books.toscrape.com...")
    books_data = scrape_books()
    
    if not books_data:
        print("No books data found or error occurred.")
        return
    
    # Get target currency from user
    print("\nAvailable currencies: USD, EUR, KES")
    target_currency = input("Enter currency to convert to (default: KES): ").strip().upper() or "KES"
    
    # Validate currency input
    valid_currencies = ['USD', 'EUR', 'KES']
    if target_currency not in valid_currencies:
        print(f"Invalid currency. Defaulting to KES.")
        target_currency = "KES"
    
    print(f"\nConverting prices from GBP to {target_currency} using Post4CSV API...")
    
    # Convert prices using Post4CSV API
    for book in books_data:
        book['price_converted'] = post4csv_convert(book['price_gbp'], 'GBP', target_currency)
        book['currency'] = target_currency
        book['converted_at'] = datetime.now().isoformat()
    
    # Display results
    display_results(books_data, target_currency)
    
    # Save to files
    save_to_csv(books_data)
    save_to_json(books_data)
    
    print(f"\nConversion done at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if __name__ == "__main__":
    main()

Scraping book prices from books.toscrape.com...

Available currencies: USD, EUR, KES

Converting prices from GBP to KES using Post4CSV API...
API request failed: HTTPSConnectionPool(host='api.post4csv.com', port=443): Max retries exceeded with url: /convert?amount=51.77&from=GBP&to=KES&apikey=98f1bd1273c4e5e295f59f5a (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E7902BF750>: Failed to resolve 'api.post4csv.com' ([Errno 11001] getaddrinfo failed)"))
Using fallback conversion for 51.77 GBP to KES
API request failed: HTTPSConnectionPool(host='api.post4csv.com', port=443): Max retries exceeded with url: /convert?amount=53.74&from=GBP&to=KES&apikey=98f1bd1273c4e5e295f59f5a (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E7903AC7D0>: Failed to resolve 'api.post4csv.com' ([Errno 11001] getaddrinfo failed)"))
Using fallback conversion for 53.74 GBP to KES
API request failed: HTTPSConnectionPool(host='api.post4csv.com